## Deliverable 2. Create a Customer Travel Destinations Map.

In [40]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [41]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df = city_data_df.drop('Unnamed: 0', axis=1)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Yellowknife,CA,62.4560,-114.3525,46.92,100,90,4.61,light rain
1,Casablanca,MA,33.5928,-7.6192,70.14,94,40,6.91,scattered clouds
2,Ushuaia,AR,-54.8000,-68.3000,38.86,70,0,10.36,clear sky
3,Kahului,US,20.8947,-156.4700,77.02,75,20,13.80,few clouds
4,Khani,GE,41.9563,42.9566,58.57,78,100,3.65,overcast clouds


In [42]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [43]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]

In [44]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   257
Country                255
Lat                    257
Lng                    257
Max Temp               257
Humidity               257
Cloudiness             257
Wind Speed             257
Current Description    257
dtype: int64

In [45]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City                   255
Country                255
Lat                    255
Lng                    255
Max Temp               255
Humidity               255
Cloudiness             255
Wind Speed             255
Current Description    255
dtype: int64

In [46]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kahului,US,77.02,few clouds,20.8947,-156.4700,
15,Laguna,US,77.31,clear sky,38.4210,-121.4238,
20,Cabo San Lucas,MX,84.51,clear sky,22.8909,-109.9124,
22,Palmas,BR,84.07,few clouds,-10.2128,-48.3603,
25,Esna,EG,84.22,clear sky,25.2934,32.5540,
28,Kruisfontein,ZA,78.40,overcast clouds,-34.0033,24.7314,
31,Sao Filipe,CV,78.26,scattered clouds,14.8961,-24.4956,
32,Atuona,PF,78.04,light rain,-9.8000,-139.0333,
33,Shu,KZ,75.90,clear sky,43.5983,73.7614,
38,Port Hedland,AU,84.25,overcast clouds,-20.3167,118.5667,


In [47]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
print('Data Retrieval Complete')        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Data Retrieval Complete


In [48]:
# 7. Drop the rows where there is no Hotel Name.

# Get names of indexes for which column Hotel Name has empty strings ("")
indexNames = hotel_df[hotel_df['Hotel Name'] == ""].index
# Delete these row indexes from dataFrame
hotel_df.drop(indexNames , inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kahului,US,77.02,few clouds,20.8947,-156.4700,Maui Seaside Hotel
15,Laguna,US,77.31,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
20,Cabo San Lucas,MX,84.51,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
22,Palmas,BR,84.07,few clouds,-10.2128,-48.3603,Hotel Lago da Palma
25,Esna,EG,84.22,clear sky,25.2934,32.5540,Orient Star Dahabyia Isna 05 Nights Each Sunday


In [49]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))